In [3]:
import logging
logger = logging.getLogger("spacy")
logger.setLevel(logging.ERROR)

import dask.bag as db
import json
import pandas as pd

docs = db.read_text('/Users/c.filtz/Downloads/arxiv-metadata-oai-snapshot.json').map(json.loads)

In [4]:
# start prototyping with a subset of the data so it's easyer to handel:
# This procedure was recommended in the ArXiv dataset itself

get_latest_version = lambda x: x['versions'][-1]['created']


# get only necessary fields of the metadata file
trim = lambda x: {'id': x['id'],
                  'authors': x['authors'],
                  'title': x['title'],
                  'doi': x['doi'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'],}
# filter for papers published on or after 2019-01-01
columns = ['id','category','abstract']
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2018)
           .map(trim).
           compute())

# convert to pandas
docs_df = pd.DataFrame(docs_df)

In [5]:
#save trimmed dataset for later use so we can skip the dataset trimming later:
docs_df.to_csv("trimmed_arxiv_docs.csv", index=False)

In [7]:
docs_df.head(10)

,id,authors,title,doi,category,abstract
0,0704.0479,T.Geisser,The affine part of the Picard scheme,None,"[math.AG, math.KT]",We describe the maximal torus and maximal un...
1,0704.1445,Yasha Gindikin and Vladimir A. Sablikov,Deformed Wigner crystal in a one-dimensional q...,10.1103/PhysRevB.76.045122,"[cond-mat.str-el, cond-mat.mes-hall]",The spatial Fourier spectrum of the electron...
2,0705.0033,"Nikos Frantzikinakis, Randall McCutcheon",Ergodic Theory: Recurrence,None,[math.DS],We survey the impact of the Poincar\'e recur...
3,0705.0344,J. P. Pridham,Unifying derived deformation theories,None,[math.AG],We develop a framework for derived deformati...
4,0705.0825,Ram Gopal Vishwakarma (Zacatecas University),Einstein's Theory of Gravity in the Presence o...,10.1007/s10509-009-0016-8,"[gr-qc, astro-ph, hep-th]",The mysterious `dark energy' needed to expla...
5,0705.2562,Ken D. Olum and Delia Schwartz-Perlov,Anthropic prediction in a large toy landscape,10.1088/1475-7516/2007/10/010,[hep-th],The successful anthropic prediction of the c...
6,0705.3164,Gabriel Vigny,Dirichlet-like space and capacity in complex a...,None,"[math.CV, math.FA]","For a Kahler manifold X, we study a space of..."
7,0705.4004,Samy Skander Bahoura (IMJ),About Brezis-Merle Problem with Lipschitz cond...,None,[math.AP],We give blow-up analysis for a Brezis-Merle'...
8,0705.4060,Ruy Exel and Artur O. Lopes,C*- Algebras and Thermodynamic Formalism,None,"[math.DS, math.OA]",We present a detailed exposition (for a Dyna...
9,0705.4151,"Maria Deijfen, Henri van den Esker, Remco van ...",A preferential attachment model with random in...,10.1007/s11512-007-0067-4,"[math.PR, math.CO]","In this paper, a random graph process ${G(t)..."


In [10]:
docs_df

,id,authors,title,doi,category,abstract
0,0704.0479,T.Geisser,The affine part of the Picard scheme,None,"[math.AG, math.KT]",We describe the maximal torus and maximal un...
1,0704.1445,Yasha Gindikin and Vladimir A. Sablikov,Deformed Wigner crystal in a one-dimensional q...,10.1103/PhysRevB.76.045122,"[cond-mat.str-el, cond-mat.mes-hall]",The spatial Fourier spectrum of the electron...
2,0705.0033,"Nikos Frantzikinakis, Randall McCutcheon",Ergodic Theory: Recurrence,None,[math.DS],We survey the impact of the Poincar\'e recur...
3,0705.0344,J. P. Pridham,Unifying derived deformation theories,None,[math.AG],We develop a framework for derived deformati...
4,0705.0825,Ram Gopal Vishwakarma (Zacatecas University),Einstein's Theory of Gravity in the Presence o...,10.1007/s10509-009-0016-8,"[gr-qc, astro-ph, hep-th]",The mysterious `dark energy' needed to expla...
...,...,...,...,...,...,...
537791,quant-ph/0612050,"Igor Devetak, Jon Yard",The exact cost of redistributing multipartite ...,10.1103/PhysRevLett.100.230501,[quant-ph],How correlated are two quantum systems from ...
537792,quant-ph/0701163,Daegene Song,Does Observation Create Reality?,None,[quant-ph],It has been suggested that the locality of i...
537793,quant-ph/0702160,"Andrew M. Childs, Richard Cleve, Stephen P. Jo...",Discrete-query quantum algorithm for NAND trees,10.4086/toc.2009.v005a005,[quant-ph],"Recently, Farhi, Goldstone, and Gutmann gave..."
537794,quant-ph/9606017,Arthur Jabs,Quantum Mechanics in Terms of Realism,None,[quant-ph],.We expound an alternative to the Copenhagen...
